In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.keras.layers import(SimpleRNN,Embedding,Input,LSTM,Input,
                                    Dropout,Dense,GRU,LayerNormalization,
                                    Bidirectional,Reshape)
from tensorflow.data.experimental import AUTOTUNE
import numpy as np
import re
import string
import nltk
import datetime
import numpy as np
from matplotlib import pyplot as plt
import pandas

<h1>Utils</h1>

In [ ]:
with open('...') as f:
    data=json.load(f)
def get_position(a,b):
    b=list(b)
    if '' in b:
        b.remove('')
    j=0
    found=0
    for i in range(len(a)):
        if b[j]==a[i]:
            j+=1
            if len(b)-1 <j:
                found=i
                break
        else:
            j=0
    return found-len(b)+1,found

f=open('...','a',encoding='utf-8')

for i in range(1):
    for j in range(1):
        for k in range(len(data["data"][i]["paragraphs"][j]["qas"])):
            if data["data"][i]["paragraphs"][j]["qas"][k]["answers"]==[]:
                pass
            else:
                init,final=get_position(tf.strings.split(preprocess_sentences(data["data"][i]["paragraphs"][j]["context"])),
                                tf.strings.split(preprocess_sentences(data["data"][i]["paragraphs"][j]["qas"][k]["answers"][0]['text'])))
            
                f.write(data["data"][i]["paragraphs"][j]["context"])
                f.write("\t")
                
                f.write(data["data"][i]["paragraphs"][j]["qas"][k]["question"])
                f.write("\t")
                
                f.write(str(init))
                f.write("\t")
                
                f.write(str(final))
                f.write("\n")
    if i%10==0:
        print(i)
f.close()

<H1>DATA PREPARATION</H1>

In [ ]:
with open('...') as f:
    data=json.load(f)

In [ ]:
path='...'

In [ ]:
NUM_EXAMPLES=1000
VALIDATION_RATIO=1
VALIDATION_BRIDGE=int(VALIDATION_RATIO*NUM_EXAMPLES)

text_dataset=tf.data.TextLineDataset(path).take(NUM_EXAMPLES)
BATCH_SIZE=1024

In [ ]:
def selector(input_text):
    return tf.strings.split(input_text,'\t')[0:1],tf.strings.split(input_text,'\t')[1:2],tf.strings.split(input_text,'\t')[2:3],tf.strings.split(input_text,'\t')[3:4],tf.strings.split(input_text,'\t')[0:2]

In [ ]:
text_dataset=text_dataset.map(selector)

In [ ]:
for i in text_dataset.take(1):
    print(i)

In [ ]:
def preprocess_sentences(input_data):
    '''
    Task: Preprocess sentences or standardize the sentences
    Input: raw reviews
    output: standardized reviews
    '''
    output=tf.strings.lower(input_data)
    outputs=tf.strings.regex_replace(output,"<[^>]+>","")
    outputs=tf.strings.regex_replace(output,"<[%s]"%re.esceape(string.punctuation)," ")
    outputs=tf.strings.regex_replace(output,"  "," ")
    
    return output

In [ ]:
CONTEXT_LENGTH=150
QUESTION_LENGTH=20

In [ ]:
vectorize_layer=TextVectorization(
    standardize=preprocess_sentences,
    output_sequence_length=CONTEXT_LENGTH,
)

In [ ]:
training_data=text_dataset.map(lambda x,y,z,w,h:h)
vectorize_layer.adapt(training_data)

In [ ]:
VOCAB_INPUT_SIZE=len(vectorize_layer.get_vocabulary())

In [ ]:
def vectorizer(x,y,z,w,h):
    return {'in1':tf.squeeze(vectorize_layer(x),0),'in2':tf.squeeze(vectorize_layer(y),0)[0:QUESTION_LENGTH],}, {'out1':tf.cast(tf.squeeze(tf.one_hot(int(z),CONTEXT_LENGTH),0),tf.int32),'out2':tf.cast(tf.squeeze(tf.one_hot(int(w),CONTEXT_LENGTH),0),tf.int32)}
dataset=text_dataset.map(vectorizer)

In [ ]:
for i,j in dataset.take(1):
    print(i,j)

In [ ]:
dataset=dataset.shuffle(NUM_EXAMPLES)
train_dataset=dataset.take(VALIDATION_BRIDGE)
validation_dataset=dataset.skip(VALIDATION_BRIDGE)


In [ ]:
train_dataset=train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
validation_dataset=validation_dataset.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)


<H1>MODELING</H1>

In [ ]:
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self,model_size):
        super(SelfAttention,self).__init__()
        self.model_size=model_size
    def call(self,query,key,value,sequence=None):
        score=tf.einsum('ijk,ibk->ijb',query,key)
        score/=tf.math.sqrt(tf.cast(self.model_size,tf.float32))
        
        if sequence is not None:
            ones=tf.ones_like(score)
            pad_mask=padding_mask(sequence)

            total_mask=pad_mask
            score+=total_mask*-1e10
        alignment=tf.nn.softmax(score,axis=-1)
        head=tf.matmul(alignment,value)
        return head

In [ ]:
def padding_mask(a):
    return tf.expand_dims(tf.cast(tf.math.equal([a],0),tf.float32)[0],axis=-2)

In [ ]:
def positional_embedding(model_size):
    output=[]
    for pos in range(SEQUENCE_LENGTH):
        PE=np.zeros((model_size))
        for i in range(model_size):
            if i%2==0:
                PE[i]=np.sin(pos/(10000**(i/model_size)))
            else:
                PE[i]=np.cos(pos/(10000**((i-1)/model_size)))
        output.append(tf.expand_dims(PE,axis=0))
    return tf.concat(output,axis=0)

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,model_size,h):
        super(MultiHeadAttention,self).__init__()
        self.query_size=model_size//h
        self.key_size=model_size//h
        self.value_size=model_size//h
        self.h=h
        self.dense_q=[Dense(self.query_size) for _ in range(h)]
        self.dense_k=[Dense(self.key_size) for _ in range(h)]
        self.dense_v=[Dense(self.value_size) for _ in range(h)]
        self.dense_o=Dense(model_size)
        self.self_attention=SelfAttention(self,key_size)
        
    def call(self,query,key,value,sequence=None):
        heads=[]
        
        for i in range(self.h):
            head=self.self_attention(self.dense_q[i](query),self.dense_k[i](key),
                                    self.dense_v[i](value),sequence)
            heads.append(head)
        heads=tf.concat(heads,axis=2)
        heads=self.dense_o(heads)
        return heads

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self,vocab_size,model_size,h):
        super(EncoderLayer,self).__init__()
        
        self.model_size=model_size
        self.h=h
        self.embedding=Embedding(vocab_size,model_size)
        self.multi_attention=MultiHeadAttention(model_size,h)
        self.dropout=Dropout(0.2)
        
        self.dense_1=Dense(model_size*4,activation='relu')
        self.dense_2=Dense(model_size)
        self.feed_forward_norm=LayerNormalization()
        
    def call(self,enc_in,sequence=None):
        enc_out=self.multi_attention(enc_in,enc_in,enc_in,sequence,)
        enc_out=enc_in+enc_out
        enc_out=self.attention_norm(enc_out)
        
        feed_forward_in=enc_out
        feed_forward_out=self.dropout(self.dense_2(self.dense_1(feed_forward_in)))
        feed_forward_out+=feed_forward_in
        feed_forward_out=self.feed_forward_norm(feed_forward_out)
        return feed_forward_out

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self,vocab_size,model_size,h,num_layers):
        super(Encoder,self).__init__()
        
        self.model_size=model_size
        self.num_layers=num_layers
        self.h=h
        self.embedding=Embedding(vocab_size,model_size)
        self.encoder_layer=[EncoderLayer(vocab_size,model_size,h) for _ in range(num_layers)]
        
    def call(self, sequence):
        enc_in=self.embedding(sequence)
        
        for i in range(self.num_layers):
            out=self.encoder_layer[i](enc_in,sequence)
            enc_in=out
        return out

In [ ]:
class EncoderTop(tf.keras.layers.Layer):
    def __init__(self,vocab_size,model_size,h,num_layers):
        super(EncoderTop,self).__init__()
        
        self.model_size=model_size
        self.num_layers=num_layers
        self.h=h
        self.embedding=Embedding(vocab_size,model_size)
        self.encoder_layer=[EncoderLayer(vocab_size,model_size,h) for _ in range(num_layers)]
        
    def call(self, enc_in):
        
        for i in range(self.num_layers):
            out=self.encoder_layer[i](enc_in,None)
            enc_in=out
        return out

In [ ]:
class CQAttention(tf.keras.layers.Layer):
    def __init__(self,vocab_size,model_size,h,num_layers):
        super(CQAttention,self).__init__()
        
        
    def call(self, enc_in):
        score=tf.einsum('BNC,BCD->BND',context,query)
        score=tf.nn.softmax(score,axis=-2)
        
        return score

In [ ]:
MODEL_SIZE=256
NUM_HEADS=8
NUM_LAYERS=3

in_1=Input(CONTEXT_LENGTH)
in_1=Input(QUESTION_LENGTH)

enc_1_out=Encoder(VOCAB_INPUT_SIZE,MODEL_SIZE,NUM_HEADS,NUM_LAYERS)(in_1)
enc_2_out=Encoder(VOCAB_INPUT_SIZE,MODEL_SIZE,NUM_HEADS,NUM_LAYERS)(in_2)

out=CQAttention()(enc_1_out,enc_2_out)

out_1=EncoderTop(VOCAB_INPUT_SIZE,MODEL_SIZE,NUM_HEADS,NUM_LAYERS)(out)
out_2=EncoderTop(VOCAB_INPUT_SIZE,MODEL_SIZE,NUM_HEADS,NUM_LAYERS)(out_1)
out_3=EncoderTop(VOCAB_INPUT_SIZE,MODEL_SIZE,NUM_HEADS,NUM_LAYERS)(out_2)
out_4=EncoderTop(VOCAB_INPUT_SIZE,MODEL_SIZE,NUM_HEADS,NUM_LAYERS)(out_3)

head_start=tf.concat([out_1,out_2,out_3],axis=-1)
head_end=tf.concat([out_1,out_4],axis=-1)

head_start=tf.nn.softmax(Dense(1)(head_start),axis=-2)
head_end=tf.nn.softmax(Dense(1)(head_end),axis=-2)

out_4=EncoderTop(VOCAB_INPUT_SIZE,MODEL_SIZE,NUM_HEADS,NUM_LAYERS)(out_3)

out={"out1":tf.squeeze(head_start,axis=-1),"out2":tf.squeeze(head_end,-1)}
model=tf.keras.Model([in_1,in_2],out,name='CQANet')
model.summary()

<H1>TRAINING</H1>

In [ ]:
def f1_score(y_true,y_pred):
    precision=tf.keras.metrics.Precision()(y_true,y_pred)
    recall=tf.keras.metrics.Recall()(y_true,y_pred)
    return 2*precision*recall/(precision+recall+1e-6)

In [ ]:
LR=1e-4
EPOCH=1000
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=LR,),
    metrics=f1_score,
    run_eagerly=True,
)

In [ ]:
checkpoint_filepath='...'
log_dir='...'
callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True
)

In [ ]:
history=model.fit(train_dataset,verbose=1,epochs=EPOCH,callbacks=[callback])

<H1>TESTING</H1>

In [ ]:
context="..."
question="..."

In [ ]:
test_data_1=tf.data.Dataset.from_tensor_slices([[context]])
test_data_2=tf.data.Dataset.from_tensor_slices([[question]])

input_test_data_1=test_data_1.map(vectorize_layer)
input_test_data_2=test_data_2.map(vectorize_layer)

for i in input_test_data_1.take(1):
    in_1=i
for i in input_test_data_2.take(1):
    in_2=i[:,0:QUESTION_LENGTH]
    
start=tf.argmax(model.predict([in_1,in_2])['out1'],-1)[0].numpy()
end=tf.argmax(model.predict([in_1,in_2])['out2'],-1)[0].numpy()

out=tf.strings.split(preprocess_sentences(context),' ')[start:end+1]

answer=""
for i in out:
    answer+=i
    answer+=" "
print(answer)

<h1>LSHAttention</h1>

In [ ]:
def look_one_back(x):
    x_extra=tf.concat([x[:,-1:,...],x[:,:-1,...]],axis=1)
    return tf.concat([x,x_extra],axis=2)

In [ ]:
def sticker_look_one_back(x):
    x_extra=tf.concat([x[:-1:],x[:,:-1]],axis=1)
    return tf.concat([x,x_extra],axis=-1)

In [ ]:
def causal_masker(a,b):
    a,b=tf.cast(a,dtype=tf.float32)+0.01,tf.cast(b,dtype=tf.float32)+0.01
    vals=tf.einsum('ipj,ipk->ipjk',b,1/a)
    out=tf.cast(tf.cast(tf.cast(vals,dtype=tf.int32),dtype=tf.bool),dtype=tf.int32)
    out=-out+1
    return tf.cast(out,dtype=tf.float32)

In [ ]:
class LSHAttention(tf.keras.layers.Layer):
    def __init__(self,bucket_size=8,n_hashes=1):
        super(LSHAttention,self).__init__()
        self.n_hashes=n_hashes
        self.bucket_size=bucket_size
        
    def call(self,query,key,value,causal_masking=False):
        R=tf.random.normal((tf.shape(query)[0],tf.shape(query)[-1],self.bucket_size//2))
        xR=tf.matmul(query,R)
        concat_xR=tf.concat([xR,-xR],axis=-1)
        buckets=tf.math.argmax(concat_xR,axis=-1)
        
        sticker=tf.argsort(buckets)
        undo_sort=tf.argsort(sticker)
        sorted_query=tf.gather(query,sticker,axis=1,batch_dims=1)
        sorted_value=tf.gather(value,sticker,axis=1,batch_dims=1)
        
        chunked_query=tf.stack(tf.split(sorted_query,self.bucket_size,1),1)
        chunked_value=tf.stack(tf.split(sorted_value,self.bucket_size,1),1)
        
        sticker=tf.stack(tf.split(sticker,self.bucket_size,1),1)
        new_sticker=sticker_look_ne_back(sticker)
        
        lb_chunked_query=look_one_back(chunked_query)
        lb_chunked_value=look_one_back(chunked_value)
        
        score=tf.einsum('bhie,bhje->bhij',chunked_query,lb_chunked_query)
        score/=tf.math.sqrt(tf.cast(query.shape[-1],tf.float32))
        
        if causal_masking==True:
            causal_mask=causal_masker(sticker,new_sticker)
            dots+=causal_mask*-1e-10
        score=tf.nn.softmax(score)
        output=tf.einsum('buij,buje->buie',score,lb_chunked_value)
        
        sorted_output=tf.reshape(output,(tf.shape(output)[0],tf.shape(query)[i],output.shape[3]))
        output=tf.gather(sorted_output,undo_sort,axis=1,batch_dims=1)
        return output